<a href="https://colab.research.google.com/github/Rokdahlia/PROGCOM-A/blob/main/TARJETA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:


import os
import json
import tempfile
import shutil
from datetime import datetime
from getpass import getpass

CARDS_DIR = "cards"

def ensure_cards_dir():
    if not os.path.isdir(CARDS_DIR):
        os.makedirs(CARDS_DIR, exist_ok=True)

def atomic_write_json(path, data):
    """Escribe JSON de forma atómica (escribe a archivo temporal y renombra)."""
    dirn = os.path.dirname(path)
    fd, tmp_path = tempfile.mkstemp(dir=dirn)
    try:
        with os.fdopen(fd, "w", encoding="utf-8") as f:
            json.dump(data, f, indent=4, ensure_ascii=False)
        shutil.move(tmp_path, path)
    finally:
        if os.path.exists(tmp_path):
            os.remove(tmp_path)

def card_filepath(card_number):
    ensure_cards_dir()
    safe_name = card_number.replace(" ", "").replace("-", "")
    return os.path.join(CARDS_DIR, f"{safe_name}.json")

class Tarjeta:
    def __init__(self, numero_tarjeta, nombre_titular, fecha_expiracion,
                 codigo_seguridad, clave, codigo_asociado="", saldo=0.0,
                 bloqueada=False, intentos_fallidos=0, transacciones=None):
        self.numero_tarjeta = str(numero_tarjeta)
        self.nombre_titular = nombre_titular
        self.fecha_expiracion = fecha_expiracion
        self.codigo_seguridad = str(codigo_seguridad)
        self.clave = str(clave)
        self.codigo_asociado = str(codigo_asociado)
        self.saldo = float(saldo)
        self.bloqueada = bool(bloqueada)
        self.intentos_fallidos = int(intentos_fallidos)
        self.transacciones = transacciones if transacciones is not None else []

    @classmethod
    def cargar(cls, numero_tarjeta):
        path = card_filepath(numero_tarjeta)
        if not os.path.exists(path):
            raise FileNotFoundError("No existe tarjeta con ese número.")
        with open(path, "r", encoding="utf-8") as f:
            data = json.load(f)
        return cls(**data)

    def guardar(self):
        path = card_filepath(self.numero_tarjeta)
        data = {
            "numero_tarjeta": self.numero_tarjeta,
            "nombre_titular": self.nombre_titular,
            "fecha_expiracion": self.fecha_expiracion,
            "codigo_seguridad": self.codigo_seguridad,
            "clave": self.clave,
            "codigo_asociado": self.codigo_asociado,
            "saldo": self.saldo,
            "bloqueada": self.bloqueada,
            "intentos_fallidos": self.intentos_fallidos,
            "transacciones": self.transacciones
        }
        atomic_write_json(path, data)

    def registrar_transaccion(self, tipo, monto, detalle=""):
        ahora = datetime.utcnow().isoformat() + "Z"
        trans = {
            "timestamp": ahora,
            "tipo": tipo,
            "monto": float(monto),
            "detalle": detalle,
            "saldo_resultante": self.saldo
        }
        self.transacciones.append(trans)

        self.guardar()

    def verificar_clave(self, intento_clave):
        if self.bloqueada:
            return False, "Tarjeta bloqueada."
        if str(intento_clave) == self.clave:
            self.intentos_fallidos = 0
            self.guardar()
            return True, "Clave correcta."
        else:
            self.intentos_fallidos += 1
            if self.intentos_fallidos >= 3:
                self.bloqueada = True
                msg = "Clave incorrecta. Tarjeta bloqueada por 3 intentos fallidos."
            else:
                msg = f"Clave incorrecta. Intentos fallidos: {self.intentos_fallidos}/3"
            self.guardar()
            return False, msg

    def retirar(self, monto):
        monto = float(monto)
        if monto <= 0:
            return False, "Monto inválido."
        if monto > self.saldo:
            return False, "Saldo insuficiente."
        self.saldo -= monto
        self.registrar_transaccion("retiro", monto, detalle="Retiro ATM")
        return True, f"Retirado ${monto:.2f}. Nuevo saldo: ${self.saldo:.2f}"

    def depositar(self, monto):
        monto = float(monto)
        if monto <= 0:
            return False, "Monto inválido."
        self.saldo += monto
        self.registrar_transaccion("deposito", monto, detalle="Depósito ATM")
        return True, f"Depositado ${monto:.2f}. Nuevo saldo: ${self.saldo:.2f}"

    def consultar_saldo(self):
        self.registrar_transaccion("consulta", 0.0, detalle="Consulta de saldo")
        return self.saldo

    def bloquear(self):
        self.bloqueada = True
        self.guardar()

    def desbloquear(self):
        self.bloqueada = False
        self.intentos_fallidos = 0
        self.guardar()

def crear_tarjeta_interactiva():
    print("=== Crear nueva tarjeta ===")
    numero = input("Número de tarjeta (16 dígitos, sin espacios): ").strip()
    nombre = input("Nombre del titular: ").strip()
    fecha = input("Fecha expiración (MM/AA): ").strip()
    cvv = input("CVV (3 dígitos): ").strip()
    clave = getpass("PIN (4 dígitos, no se muestra): ").strip()
    codigo = input("Código asociado (opcional, p.ej 'QST-2025-XXXX'): ").strip()
    saldo_inicial = input("Saldo inicial (ej. 1000.50): ").strip() or "0"
    tarjeta = Tarjeta(numero, nombre, fecha, cvv, clave, codigo, saldo=float(saldo_inicial))
    tarjeta.guardar()
    print(f"Tarjeta creada y guardada en {card_filepath(numero)}")
    return tarjeta

def listar_tarjetas():
    ensure_cards_dir()
    archivos = [f for f in os.listdir(CARDS_DIR) if f.lower().endswith(".json")]
    if not archivos:
        print("No hay tarjetas registradas.")
        return
    print("Tarjetas guardadas:")
    for a in archivos:
        print(" -", a)

def insertar_tarjeta():
    print("=== Insertar tarjeta en el cajero ===")
    numero = input("Introduce número de tarjeta: ").strip()
    try:
        tarjeta = Tarjeta.cargar(numero)
    except FileNotFoundError:
        print("Tarjeta no encontrada. Puedes crearla desde el menú principal.")
        return

    if tarjeta.bloqueada:
        print("⚠️  Tarjeta bloqueada. Contacte con el banco.")
        return


    for intento in range(3):
        pin = getpass("Introduce PIN (no se muestra): ").strip()
        ok, msg = tarjeta.verificar_clave(pin)
        print(msg)
        if ok:
            break
        if tarjeta.bloqueada:
            return
    if not ok:

        return


    opciones_avanzadas_habilitadas = False
    if tarjeta.codigo_asociado:
        usar_codigo = input("¿Desea ingresar el código asociado para habilitar opciones avanzadas? (s/n): ").strip().lower()
        if usar_codigo == "s":
            codigo_intento = input("Ingrese código asociado: ").strip()
            if codigo_intento == tarjeta.codigo_asociado:
                opciones_avanzadas_habilitadas = True
                print("✅ Opciones avanzadas habilitadas.")
            else:
                print("Código asociado incorrecto. Opciones avanzadas no habilitadas.")


    atm_menu(tarjeta, opciones_avanzadas_habilitadas)

def atm_menu(tarjeta, opciones_avanzadas=False):
    while True:
        print("\n--- MENÚ CAJERO ---")
        print("1) Consultar saldo")
        print("2) Retirar")
        print("3) Depósito")
        print("4) Historial de transacciones (últimas 10)")
        if opciones_avanzadas:
            print("5) Opciones avanzadas")
            print("6) Finalizar sesión")
            maxopt = "6"
        else:
            print("5) Finalizar sesión")
            maxopt = "5"

        opcion = input("Seleccione una opción: ").strip()

        if opcion == "1":
            saldo = tarjeta.consultar_saldo()
            print(f"Saldo actual: ${saldo:.2f}")
        elif opcion == "2":
            monto = input("Monto a retirar: ").strip()
            try:
                monto_f = float(monto)
            except ValueError:
                print("Monto inválido.")
                continue
            ok, msg = tarjeta.retirar(monto_f)
            print(msg)

        elif opcion == "3":
            monto = input("Monto a depositar: ").strip()
            try:
                monto_f = float(monto)
            except ValueError:
                print("Monto inválido.")
                continue
            ok, msg = tarjeta.depositar(monto_f)
            print(msg)

        elif opcion == "4":
            mostrar_historial(tarjeta)

        elif opciones_avanzadas and opcion == "5":
            menu_avanzado(tarjeta)

        elif (not opciones_avanzadas and opcion == "5") or (opciones_avanzadas and opcion == "6"):
            print("Cerrando sesión. Retire su tarjeta.")
            return

        else:
            print("Opción no válida. Intente de nuevo.")

def mostrar_historial(tarjeta, n=10):
    print(f"Últimas {n} transacciones (más recientes primero):")
    hist = tarjeta.transacciones[-n:]
    if not hist:
        print("No hay transacciones.")
        return
    for t in reversed(hist):
        ts = t.get("timestamp")
        tipo = t.get("tipo")
        monto = t.get("monto")
        saldo_res = t.get("saldo_resultante")
        detalle = t.get("detalle", "")
        print(f"- {ts} | {tipo} | ${monto:.2f} | saldo: ${saldo_res:.2f} | {detalle}")

def menu_avanzado(tarjeta):
    while True:
        print("\n--- OPCIONES AVANZADAS ---")
        print("1) Consulta extendida (datos de tarjeta)")
        print("2) Simular consulta de préstamos (demo)")
        print("3) Volver")
        opt = input("Seleccione: ").strip()
        if opt == "1":
            print("Datos de tarjeta (no mostrar CVV/PIN en producción):")
            print(f"Numero: {tarjeta.numero_tarjeta}")
            print(f"Titular: {tarjeta.nombre_titular}")
            print(f"Expiración: {tarjeta.fecha_expiracion}")
            print(f"CVV: {tarjeta.codigo_seguridad}")
            print(f"Código asociado: {tarjeta.codigo_asociado}")
            print(f"Saldo: ${tarjeta.saldo:.2f}")
        elif opt == "2":
            print("Consulta de préstamos: (demo) No hay datos reales.")
        elif opt == "3":
            return
        else:
            print("Opción no válida.")

def main_menu():
    ensure_cards_dir()
    while True:
        print("\n=== CAJERO SIMULADO ===")
        print("1) Insertar tarjeta")
        print("2) Crear nueva tarjeta")
        print("3) Listar tarjetas almacenadas")
        print("4) Salir")
        choice = input("Seleccione una opción: ").strip()
        if choice == "1":
            insertar_tarjeta()
        elif choice == "2":
            crear_tarjeta_interactiva()
        elif choice == "3":
            listar_tarjetas()
        elif choice == "4":
            print("Saliendo. ¡Hasta luego!")
            break
        else:
            print("Opción no válida.")

if __name__ == "__main__":
    main_menu()



=== CAJERO SIMULADO ===
1) Insertar tarjeta
2) Crear nueva tarjeta
3) Listar tarjetas almacenadas
4) Salir
Seleccione una opción: 1
=== Insertar tarjeta en el cajero ===
Introduce número de tarjeta: 1234567812345678
Tarjeta no encontrada. Puedes crearla desde el menú principal.

=== CAJERO SIMULADO ===
1) Insertar tarjeta
2) Crear nueva tarjeta
3) Listar tarjetas almacenadas
4) Salir
Seleccione una opción: 2
=== Crear nueva tarjeta ===
Número de tarjeta (16 dígitos, sin espacios): 1234567812345678
Nombre del titular: Lucas
Fecha expiración (MM/AA): 01/01
CVV (3 dígitos): 000
PIN (4 dígitos, no se muestra): ··········
Código asociado (opcional, p.ej 'QST-2025-XXXX'): 0000
Saldo inicial (ej. 1000.50): 10000000
Tarjeta creada y guardada en cards/1234567812345678.json

=== CAJERO SIMULADO ===
1) Insertar tarjeta
2) Crear nueva tarjeta
3) Listar tarjetas almacenadas
4) Salir
Seleccione una opción: 1
=== Insertar tarjeta en el cajero ===
Introduce número de tarjeta: 1234567812345678
Introdu

/tmp/ipython-input-2838494314.py:72: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ahora = datetime.utcnow().isoformat() + "Z"


Retirado $1000.00. Nuevo saldo: $9999000.00

--- MENÚ CAJERO ---
1) Consultar saldo
2) Retirar
3) Depósito
4) Historial de transacciones (últimas 10)
5) Opciones avanzadas
6) Finalizar sesión
